# Umpire Scraping

Given a season, scrape baseball-reference.com for the homeplate umpire

In [1]:
import csv
import requests
from bs4 import BeautifulSoup, Comment

In [2]:
schedule_page = requests.get("https://www.baseball-reference.com/leagues/MLB/2016-schedule.shtml")
soup = BeautifulSoup(schedule_page.content, 'html.parser')

In [3]:
games_list = soup.find_all("p", attrs={"game"})
games_list_href = [x.find_all("a", href=True)[-1]['href'] for x in games_list]
games_list_href[-10:]

['/boxes/TOR/TOR201610190.shtml',
 '/boxes/LAN/LAN201610200.shtml',
 '/boxes/CHN/CHN201610220.shtml',
 '/boxes/CLE/CLE201610250.shtml',
 '/boxes/CLE/CLE201610260.shtml',
 '/boxes/CHN/CHN201610280.shtml',
 '/boxes/CHN/CHN201610290.shtml',
 '/boxes/CHN/CHN201610300.shtml',
 '/boxes/CLE/CLE201611010.shtml',
 '/boxes/CLE/CLE201611020.shtml']

In [4]:
bbref_addr = 'https://www.baseball-reference.com'

hp_umpire_dict = {}
for i, game in enumerate(games_list_href):
    
    if i % 100 == 0:
        print(f"Game {i}")
        
    # get the id for the game
    game_id = game.split('/')[-1][:-7]
        
    # build the url for the game
    game_addr = bbref_addr + game
    
    # scrape the webpage for the game
    game_page = requests.get(game_addr)
    soup = BeautifulSoup(game_page.content, 'html.parser')
    
    # the umpires are stashed in a comments section
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    ump_string = [x for x in comments if 'Umpires' in x][0]
    ump_string = ump_string.strip()
    ump_string = ump_string.replace(',', '')
    
    # extract the HP umpire
    ump_list = ump_string.split()
    hp_idx = ump_list.index('HP')
    first_name = ump_list[hp_idx+2]
    last_name = ump_list[hp_idx+3]
    
    hp_umpire_dict[game_id] = first_name + ' ' + last_name
    
w = csv.writer(open("../data/hp_umpires_2016.csv", "w"))
for key, val in hp_umpire_dict.items():
    w.writerow([key, val])

Game 0
Game 100
Game 200
Game 300
Game 400
Game 500
Game 600
Game 700
Game 800
Game 900
Game 1000
Game 1100
Game 1200
Game 1300
Game 1400
Game 1500
Game 1600
Game 1700
Game 1800
Game 1900
Game 2000
Game 2100
Game 2200
Game 2300
Game 2400


In [6]:
w = csv.writer(open("../data/hp_umpires_2016.csv", "w"))
for key, val in hp_umpire_dict.items():
    w.writerow([key, val])

In [7]:
import pandas as pd
df = pd.read_csv("../data/hp_umpires_2016.csv", header=None)
df.shape

(2449, 2)